In [ ]:
import math
import pickle

import geopandas as gpd
import igraph
import osmnx as ox
from geojson import LineString
from matplotlib import pyplot as plt
from shapely.affinity import translate

from config import CPH_AMAGER_DANGER_ZONE
from data_loader import load_json_file_to_str
from data_loader.danger_zones import load_danger_zone_from_str
from data_loader.osm import load_osm
from routes.fastest_path import FastestPath
from routes.route import Route
from routes.route_algo import RouteAlgo
from routes.shortest_path import ShortestPath

In [ ]:
# Load the OSM data for Copenhagen
graph = load_osm("copenhagen.graphml")
nodes, edges = ox.graph_to_gdfs(graph)

In [ ]:
# Compute paths to safety
origin = 16965546
danger_zones = load_danger_zone_from_str(
    load_json_file_to_str(CPH_AMAGER_DANGER_ZONE), "EPSG:4326"
)


def pathfind(algorithm: RouteAlgo) -> tuple[list[str], gpd.GeoSeries]:
    path = algorithm.route_to_safety([origin], danger_zones, graph)[origin][0]
    path_coordinates = [
        (graph.nodes[node]["x"], graph.nodes[node]["y"]) for node in path
    ]
    return path, gpd.GeoSeries([LineString(path_coordinates)], crs=edges.crs)


fastest_path, fastest_line = pathfind(FastestPath())
_, shortest_line = pathfind(ShortestPath())

In [ ]:
with open("routes/amager-polaris-1_igraph.pkl", "rb") as f:
    polaris_graph: igraph.Graph = pickle.load(f)
with open("routes/amager-polaris-1_polaris_paths.pkl", "rb") as f:
    polaris_routes: list[Route] = pickle.load(f)

polaris_origin = str(origin)
for v in polaris_graph.vs:
    if v['name'] == polaris_origin:
        polaris_origin = v['original_id']
        break

origin_node = graph.nodes[origin]
origin_coords = origin_node["x"], origin_node["y"]

polaris_paths = []
for route in polaris_routes:
    # if route.path[0] == polaris_origin:
    #     polaris_paths.append(route.path)
    coords = polaris_graph.es[route.path[0]]["coordinates"]["from"]
    if math.isclose(coords[0], origin_coords[0], abs_tol=1e-3) and math.isclose(coords[1], origin_coords[1], abs_tol=1e-3):
        polaris_paths.append(route.path)

polaris_lines = []
for path in polaris_paths:
    path_coordinates = []
    for i, edge in enumerate(path):
        coordinates = polaris_graph.es[edge]["coordinates"]
        if i == 0:
            path_coordinates.append(coordinates["from"])
        path_coordinates.append(coordinates["to"])
    polaris_lines.append(gpd.GeoSeries([LineString(path_coordinates)], crs=edges.crs))

In [ ]:
# Project the GeoDataFrames to UTM zone 32N (EPSG:32632)
epsg = 32632
nodes = nodes.to_crs(epsg=epsg)
edges = edges.to_crs(epsg=epsg)

# Offset the lines to avoid overlap in the plot
path_gdfs = [
    fastest_line.to_crs(epsg=epsg).apply(lambda geom: translate(geom, xoff=-60)),
    shortest_line.to_crs(epsg=epsg).apply(lambda geom: translate(geom, xoff=-30)),
    polaris_lines[3].to_crs(epsg=epsg).apply(lambda geom: translate(geom, xoff=0)),
    polaris_lines[4].to_crs(epsg=epsg).apply(lambda geom: translate(geom, xoff=30)),
    polaris_lines[5].to_crs(epsg=epsg).apply(lambda geom: translate(geom, xoff=60)),
]

In [ ]:
# Plot the graph with the paths overlayed. Zoomed in on the paths with a buffer around them.
buffer = 1_000

labels = ['Fastest', 'Shortest', 'Polaris (1)', 'Polaris (2)', 'Polaris (3)']
colors = ['red', 'blue', 'green', 'yellow', 'cyan']

fig, ax = plt.subplots(figsize=(10, 10))
edges.plot(ax=ax, linewidth=1, edgecolor='gray')
# nodes.plot(ax=ax, markersize=1, color='black')
for gdf, color, label in zip(path_gdfs, colors, labels):
    gdf.plot(ax=ax, linewidth=3, color=color, label=label)

minx, miny, maxx, maxy = gpd.GeoSeries([g[0] for g in path_gdfs], crs=edges.crs).total_bounds
ax.set_xlim(minx - buffer, maxx + buffer)
ax.set_ylim(miny - buffer, maxy + buffer)

# ax.set_title("Graph with Path Overlay on Amager", fontsize=16)
ax.set_aspect('equal')
ax.legend(title="Paths")
ax.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)

plt.savefig("graph_with_paths.png", dpi=1000, bbox_inches='tight')
plt.show()